## 모델링

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
# from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import *

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM
from keras.backend import clear_session

: 

In [ ]:
!pip install ipykernel

#### 데이터 불러오기

In [3]:
path='../../data/preprocessed data/'

In [4]:
data=pd.read_csv(path+'dong_service_data.csv')
data.shape

(191994, 175)

In [5]:
# 추정매출 데이터 -> 당월_매출_금액만 남기고 모두 드랍.
drop_cols = ['당월_매출_건수', '주중_매출_금액', '주말_매출_금액',
       '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액',
       '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액',
       '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액',
       '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액', '연령대_10_매출_금액',
       '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액',
       '연령대_60_이상_매출_금액', '주중_매출_건수', '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수',
       '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수',
       '시간대_건수~06_매출_건수', '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수',
       '시간대_건수~17_매출_건수', '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '남성_매출_건수',
       '여성_매출_건수', '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수',
       '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수']
data=data.drop(columns=drop_cols,axis=1)
data.shape

(191994, 128)

In [6]:
# 19년도 데이터 제거
data = data[~data['기준_년분기_코드'].isin([20191, 20192, 20193, 20194])]
data['기준_년분기_코드'].unique()

array([20221, 20222, 20223, 20224, 20231, 20232, 20201, 20202, 20203,
       20204, 20211, 20212, 20213, 20214], dtype=int64)

In [7]:
data.isna().sum().sum()

0

In [8]:
# 행정동 이름, 업종 이름 컬럼 제거
data=data.drop(['행정동_코드_명','서비스_업종_코드_명'],axis=1)
data.shape

(166920, 126)

In [9]:
# 데이터프레임 추출 함수
# def data_filter_split(dataframe,행정동코드, 업종코드):
#     return dataframe[(dataframe['행정동_코드'] == 행정동코드) & (dataframe['서비스_업종_코드'] == 업종코드)]

In [11]:
# # 업종코드 object형 데이터 int형 변환
# data['서비스_업종_코드']=data['서비스_업종_코드'].str[2:].astype(int)
# data['서비스_업종_코드'].dtype

# 행정동_코드 int형 -> string 변환
data['행정동_코드']=data['행정동_코드'].astype(str)
data.dtypes

기준_년분기_코드       int64
행정동_코드         object
서비스_업종_코드      object
당월_매출_금액        int64
집객시설_수        float64
               ...   
수요일_유동인구_수      int64
목요일_유동인구_수      int64
금요일_유동인구_수      int64
토요일_유동인구_수      int64
일요일_유동인구_수      int64
Length: 126, dtype: object

In [10]:
# 11110515: 청운효자동 , 한식음식점: CS100001

In [15]:
# x,y 분리 , target: 당월_매출_금액
target='당월_매출_금액'
x = data.drop(target, axis = 1)
y = data.loc[:,target]

## Auto ML : pycaret

In [16]:
!pip install -U --pre pycaret

'DOSKEY'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [17]:
from pycaret.regression import *

In [21]:
# string 타입 제외하고 스케일링 후 합치기
exc=['행정동_코드', '서비스_업종_코드']
tmp_x = x.drop(exc, axis=1)


In [22]:
# data scaling : min-max
# 전체 데이터를 직접 스케일링 해주지 않으면 너무 큰값들이라서 pycaret 동작하지 않음
scaler=MinMaxScaler()
x_s_tmp=scaler.fit_transform(tmp_x)
x_s_tmp=pd.DataFrame(x_s_tmp)
x_s=pd.concat([x[exc],x_s_tmp],axis=1)

In [23]:
x_s.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)
data_s=pd.concat([x_s,y],axis=1)
data_s.shape


(191994, 126)

In [25]:
data_s.isna().sum()

행정동_코드       25074
서비스_업종_코드    25074
0            25074
1            25074
2            25074
             ...  
119          25074
120          25074
121          25074
122          25074
당월_매출_금액     25074
Length: 126, dtype: int64

In [24]:
# 매출 컬럼 제외하지 않은 데이터의 결과들
exp = setup(data = data_s,
            target = target,
            normalize = True,
            # normalize_method = 'minmax', # default는 zscore
            transformation = True,  # 전처리 기능 설정
            transform_target=True,
            fold = 5, # default는 10
            fold_shuffle=True,
            # ignore_features = ign_cols, # 제외할 컬럼
            # categorical_features = [], # one-hot encoding
            # date_features = [], # 날짜 feature를 년월일시 로 바꿔서 onehotencoding 해준다.
            session_id = 123, # random state number 지정
            use_gpu = True, # gpu 사용 옵션
            feature_selection = True, # 특징선택 자동
            feature_selection_method ='classic',
            #‘univariate’: Uses sklearn’s SelectKBest.
            #‘classic’: Uses sklearn’s SelectFromModel.
            #‘sequential’: Uses sklearn’s SequentialFeatureSelector.

            # fix_imbalance = True, # 클래스 불균형 문제 자동으로 해결. default는 False
            # fix_imbalance_method = imblearn.OverSampling.RandomOverSampler() # 오버샘플링 예시
            # 기본은 SMOTE

            # custom_pipeline = pipe, preprocess =False
            # 두 개는 세트, 사용자가 원하는 파이프라인을 구성할 수 있다.
            )

ValueError: 25074 missing values found in the target column: 당월_매출_금액. To proceed, remove the respective rows from the data. 

In [ ]:
# best model
best_model = compare_models()

# tune
tuned_model = tune_model(best_model)


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,02:04:23
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,140407962.2046,1030607718235282944.0000,981273650.2579,0.9850,0.2806,0.2986,147.9240
dt,Decision Tree Regressor,151208185.8104,4808076094811257856.0000,1776397975.0722,0.9385,0.4199,0.6079,25.3160
knn,K Neighbors Regressor,551901155.7918,20187806002171809792.0000,4360234357.5566,0.7530,0.7391,1.1985,36.9740
br,Bayesian Ridge,1332205517.0363,69465461711376785408.0000,8276210059.1308,0.1060,1.2736,6.9308,23.6380
lr,Linear Regression,1332728938.3143,69476778195720732672.0000,8276907724.6921,0.1059,1.2736,6.9260,24.9020
ridge,Ridge Regression,1332696988.1934,69476084562053349376.0000,8276865027.4513,0.1059,1.2736,6.9263,25.4220
lar,Least Angle Regression,1332729052.6993,69476783866596188160.0000,8276908104.6796,0.1059,1.2736,6.9260,23.2900
huber,Huber Regressor,1333682295.9172,69796119191780024320.0000,8296157260.6960,0.1017,1.2753,6.7591,25.5500
omp,Orthogonal Matching Pursuit,1358334147.3816,70852769216788946944.0000,8359872695.5611,0.0876,1.3149,5.7769,23.3760
en,Elastic Net,1530656176.8598,75535586985564504064.0000,8637691973.1506,0.0248,1.3869,7.7082,23.9120


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
# evaluate
evaluate_model(tuned_model)
# predict
predictions = predict_model(tuned_model, data=data_s)
# save
save_model(tuned_model, 'best_model_29d_v1')

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,1515718.6154,50636844887777.9219,7115957.0606,0.9999,0.0018,0.0004


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('target_transformation',
                  TransformerWrapperWithInverse(transformer=TargetTransformer(estimator=PowerTransformer(standardize=False)))),
                 ('numerical_imputer',
                  TransformerWrapper(include=['0', '1', '2', '3', '4', '5', '6',
                                              '7', '8', '9', '10', '11', '12',
                                              '13', '14', '15', '16', '17', '18',
                                              '19', '20', '21', '22', '23', '24',
                                              '25', '2...
                  TransformerWrapper(transformer=PowerTransformer(standardize=False))),
                 ('normalize', TransformerWrapper(transformer=StandardScaler())),
                 ('feature_selection',
                  TransformerWrapper(exclude=[],
                                     transformer=SelectFromModel(estimator=LGBMRegressor(),
                  

In [ ]:
# best model
best_model = compare_models()

# tune
tuned_model = tune_model(best_model)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,204208922.6940,159983455784566016.0000,284229832.9318,0.0747,0.0989,0.0679,0.7060
dt,Decision Tree Regressor,264415125.3167,240211610476643520.0000,365257397.3254,-0.0856,0.1271,0.0912,0.4960
gbr,Gradient Boosting Regressor,265412763.9172,240108541038375200.0000,365971969.7633,-0.0893,0.1272,0.0914,0.5580
ada,AdaBoost Regressor,260545558.7442,239634032656679744.0000,360607057.6470,-0.1103,0.1241,0.0890,0.5900
xgboost,Extreme Gradient Boosting,201178660.4833,155498464986255200.0000,260860367.2326,-0.7757,0.0896,0.0610,0.8360
rf,Random Forest Regressor,378250643.1360,254749824412364192.0000,473038247.7881,-41.9497,0.1474,0.1152,0.8420
par,Passive Aggressive Regressor,2661316949.2193,9149717899675844608.0000,2771754105.4979,-47.5376,8.3594,0.8130,0.6240
omp,Orthogonal Matching Pursuit,669337465.3833,949076486626257664.0000,859915379.6495,-162.4277,0.2020,0.1778,0.5180
lr,Linear Regression,669337418.4500,949076746406054656.0000,859915415.0936,-162.4277,0.2020,0.1778,0.5440
lar,Least Angle Regression,669337465.3833,949076486626257664.0000,859915379.6495,-162.4277,0.2020,0.1778,0.5060


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,702587699.2284,527356951404342592.0000,726193466.9249,0.4615,0.2577,0.2715
1,273389096.2898,94870116923508512.0000,308009929.9106,0.9393,0.1252,0.1098
2,766400893.9033,594745254493849856.0000,771197286.3631,0.3197,0.2358,0.2558
3,1110558418.1665,1243576967820303616.0000,1115157821.9339,-1128.7769,0.2736,0.2382
4,357411795.4585,190645125665619808.0000,436629277.1512,-0.3960,0.1335,0.1086
Mean,642069580.6093,530238883261524864.0000,671437556.4567,-225.4905,0.2052,0.1968
Std,301846935.7894,404447891141385984.0000,281798671.1128,451.6434,0.0631,0.0723


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
c:/Users/user/AppData/Local/Programs/Python/Python37/python.exe -m pip install ipykernel -U --user --force-reinstall

: 

## Auto ML : AutoGluon

In [26]:
!pip install autogluon

  Using cached scikit_learn-1.3.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
     ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
     -------------- ------------------------- 0.6/1.6 MB 12.5 MB/s eta 0:00:01
     -------------------------------- ------- 1.3/1.6 MB 13.7 MB/s eta 0:00:01
     ---------------------------------------- 1.6/1.6 MB 14.4 MB/s eta 0:00:00
  Using cached Pillow-10.1.0-cp311-cp311-win_amd64.whl.metadata (9.6 kB)
     ---------------------------------------- 0.0/41.2 kB ? eta -:--:--
     ---------------------------------------- 41.2/41.2 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/90.4 kB ? eta -:--:--
     ---------------------------------------- 90.4/90.4 kB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished wit

'DOSKEY'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Ignoring numpy: markers 'python_version < "3.8"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.8"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.9"' don't match your environment
        Using cached Cython-3.0.7-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
      ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
      ERROR: Could not find a version that satisfies the requirement numpy==1.21.3 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2

In [28]:
!python -m pip install --upgrade pip

'DOSKEY'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [29]:
!pip install numpy==1.20.3

     ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
     - -------------------------------------- 0.3/7.8 MB 3.2 MB/s eta 0:00:03
     -- ------------------------------------- 0.6/7.8 MB 4.4 MB/s eta 0:00:02
     ---- ----------------------------------- 0.9/7.8 MB 5.0 MB/s eta 0:00:02
     ----- ---------------------------------- 1.1/7.8 MB 5.1 MB/s eta 0:00:02
     --------- ------------------------------ 1.9/7.8 MB 7.2 MB/s eta 0:00:01
     ---------- ----------------------------- 2.1/7.8 MB 7.8 MB/s eta 0:00:01
     ---------- ----------------------------- 2.1/7.8 MB 7.8 MB/s eta 0:00:01
     ----------- ---------------------------- 2.2/7.8 MB 5.3 MB/s eta 0:00:02
     --------------- ------------------------ 3.0/7.8 MB 6.8 MB/s eta 0:00:01
     ----------------- ---------------------- 3.4/7.8 MB 7.2 MB/s eta 0:00:01
     -------------------- ------------------- 4.0/7.8 MB 7.5 MB/s eta 0:00:01


'DOSKEY'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [215 lines of output]
      setup.py:66: RuntimeWarning: NumPy 1.20.3 may not yet support Python 3.11.
        warnings.warn(
      Running from numpy source directory.
      setup.py:485: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
        run_build = parse_setuppy_commands()
      Cythonizing sources
      Processing numpy/random\_bounded_integers.pxd.in
      Processing numpy/random\bit_generator.pyx
      Processing numpy/random\mtrand.pyx
      Processing numpy/random\_bounded_integers.pyx.in
      Processing numpy/random\_common.pyx
      Processing numpy/random\_generator.pyx
      Processing numpy/random\_mt19937.pyx
      Processing numpy/random\_pcg64.pyx
      Processing numpy/random\_philox.pyx
      Pr

In [27]:
from autogluon.tabular import TabularDataset, TabularPredictor

ModuleNotFoundError: No module named 'autogluon'

In [ ]:
X_data, y_data = data_s.drop(target,axis=1), data_s[target]
X_train, X_valid, y_train, y_valid = train_test_split(X_data, y_data, test_size = .2, random_state = 42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
train_df = TabularDataset(train)
test_df = TabularDataset(test)

predictor = TabularPredictor(label = target, problem_type = 'regression', eval_metric = 'mse',
                             path = './') 

predictor.fit(train_data = train_df, presets = 'best_quality',
              auto_stack = True, fit_weighted_ensemble = True,
              num_bag_folds = 10, num_bag_sets = 30, num_stack_levels = 3,
              num_gpus = 1, num_cpus = 24, verbosity = 2)

In [ ]:
# Feature Importance 확인
feature_importance = predictor.feature_importance(train_df)
feature_importance

## 딥러닝 모델

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Flatten,InputLayer
# from keras.backend import clear_session

In [ ]:
data_s.shape

(26, 124)

In [ ]:
x=data_s.drop(target,axis=1)
y=data_s[target]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((20, 123), (6, 123), (20,), (6,))

In [ ]:
model_dl = Sequential()

# 입력 레이어
model_dl.add(InputLayer(input_shape=(123,)))

model_dl.add(Dense(units=2048, activation='relu'))
model_dl.add(Dense(units=1024, activation='relu'))
model_dl.add(Dense(units=512, activation='relu'))
model_dl.add(Dense(units=256, activation='relu'))
model_dl.add(Dense(units=128, activation='relu'))
model_dl.add(Dense(units=128, activation='relu'))
model_dl.add(Dense(units=16, activation='relu'))

model_dl.add(Dense(units=20000, activation='relu'))

# 출력 레이어
model_dl.add(Dense(units=1, activation='linear'))

# 모델 컴파일
model_dl.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# 모델 요약 출력
model_dl.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 2048)              253952    
                                                                 
 dense_6 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_7 (Dense)             (None, 512)               524800    
                                                                 
 dense_8 (Dense)             (None, 256)               131328    
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dense_11 (Dense)            (None, 16)               

In [ ]:
# 모델 훈련
model_dl.fit(x_train, y_train, epochs=2000, batch_size=32, validation_data=(x_test, y_test))

# 모델 평가
loss, mse = model_dl.evaluate(x_test, y_test)
rmse=np.sqrt(mse)
print(f"rmse: {rmse}")

Epoch 1/2000
1/1 [==============================] - 0s 210ms/step - loss: 6088319616155648.0000 - mse: 6088319616155648.0000 - val_loss: 10376474557153280.0000 - val_mse: 10376474557153280.0000
Epoch 2/2000
1/1 [==============================] - 0s 71ms/step - loss: 6062663092142080.0000 - mse: 6062663092142080.0000 - val_loss: 10356944267116544.0000 - val_mse: 10356944267116544.0000
Epoch 3/2000
1/1 [==============================] - 0s 90ms/step - loss: 6037204673495040.0000 - mse: 6037204673495040.0000 - val_loss: 10337646979055616.0000 - val_mse: 10337646979055616.0000
Epoch 4/2000
1/1 [==============================] - 0s 73ms/step - loss: 6011989457371136.0000 - mse: 6011989457371136.0000 - val_loss: 10318490351173632.0000 - val_mse: 10318490351173632.0000
Epoch 5/2000
1/1 [==============================] - 0s 101ms/step - loss: 5987019054383104.0000 - mse: 5987019054383104.0000 - val_loss: 10299519480627200.0000 - val_mse: 10299519480627200.0000
Epoch 6/2000
1/1 [===============

## RNN : LSTM

In [20]:
target='당월_매출_금액'
x = data_s.drop(target, axis = 1)
y = data_s.loc[:,target]
x_train_s,x_test_s,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)
x_train.shape,x_test.shape

((20, 123), (6, 123))

In [21]:
# lstm input 차원 맞춰주기
x_train_lstm = np.expand_dims(x_train_s, axis=-1)
x_test_lstm = np.expand_dims(x_test_s, axis=-1)
x_train_lstm.shape,x_test_lstm.shape

((20, 123, 1), (6, 123, 1))

In [ ]:


model_lstm = Sequential()

model_lstm.add(LSTM(units=64, activation='relu', return_sequences=True, input_shape=(123, 1)))
model_lstm.add(LSTM(units=32, activation='relu'))

# 출력 레이어
model_lstm.add(Dense(units=1, activation='linear'))

# 모델 컴파일
model_lstm.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# 모델 요약 출력
model_lstm.summary()

# training
model_lstm.fit(x_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(x_test_lstm, y_test))

loss, mse = model_dl.evaluate(x_test_lstm, y_test)
rmse=np.sqrt(mse)
print(f"rmse: {rmse}")

## Auto ML : AutoGluon

In [ ]:
!pip install autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:

X_data, y_data = data_s.drop(target,axis=1), data_s[target]
X_train, X_valid, y_train, y_valid = train_test_split(X_data, y_data, test_size = .2, random_state = 42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
train_df = TabularDataset(train)
test_df = TabularDataset(test)

predictor = TabularPredictor(label = target, problem_type = 'regression', eval_metric = 'mse',
                             path = './')

predictor.fit(train_data = train_df, presets = 'best_quality',
              auto_stack = True, fit_weighted_ensemble = True,
              num_bag_folds = 10, num_bag_sets = 30, num_stack_levels = 3,
              num_gpus = 1, num_cpus = 24, verbosity = 2)

In [ ]:
# Feature Importance 확인
feature_importance = predictor.feature_importance(train_df)
feature_importance